# Прогнозирование оттока клиента Банка

# Задача
Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Нам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Нужно построить модель с предельно большим значением *F1*-меры. Требуется довести метрику до 0.59.

# План

**1. Подготовка данных.**

**2. Исследование задачи.**

**3. Борьба с дисбалансом.**

**4. Тестирование модели.**

**5. Выводы.**

## Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings('ignore')

In [2]:
churn_data = pd.read_csv('/datasets/Churn.csv')
churn_data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [3]:
churn_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


**Промежуточные выводы**

Итак, нам нужно устранить пропуски в столбце Tenure, поправить названия столбцов, а также проверить данные на наличие дубликатов.

In [4]:
#Исправим названия столбцов
churn_data = churn_data.rename(columns={'RowNumber':'row_number', 'CustomerId':'customer_id', \
                                        'CreditScore':'credit_score', 'NumOfProducts':'num_of_products', \
                                        'HasCrCard':'has_cr_card', 'IsActiveMember':'is_active_member', \
                                        'EstimatedSalary':'estimated_salary'})
churn_data.columns = churn_data.columns.str.lower()

In [5]:
#Избавимся от пропусков в столбце tenure
#Из всех данных на пропуски приходится 9%, поэтому мы можем удалить эти строки
churn_data = churn_data.loc[churn_data['tenure'].isna() == False]

In [6]:
churn_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9091 entries, 0 to 9998
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_number        9091 non-null   int64  
 1   customer_id       9091 non-null   int64  
 2   surname           9091 non-null   object 
 3   credit_score      9091 non-null   int64  
 4   geography         9091 non-null   object 
 5   gender            9091 non-null   object 
 6   age               9091 non-null   int64  
 7   tenure            9091 non-null   float64
 8   balance           9091 non-null   float64
 9   num_of_products   9091 non-null   int64  
 10  has_cr_card       9091 non-null   int64  
 11  is_active_member  9091 non-null   int64  
 12  estimated_salary  9091 non-null   float64
 13  exited            9091 non-null   int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.0+ MB


In [7]:
#Проверим данные на наличие дубликатов
print(churn_data.duplicated().sum())
print(churn_data['geography'].unique())
print(churn_data['gender'].unique())

0
['France' 'Spain' 'Germany']
['Female' 'Male']


**Промежуточные выводы**

Мы устранили пропуски в столбце Tenure, поправили названия столбцов. Дубликаты в данных не обнаружены.

## Исследование задачи

In [8]:
#Изучим баланс классов
print(churn_data.loc[churn_data['exited'] == 0, 'exited'].count()/len(churn_data)*100)
print(churn_data.loc[churn_data['exited'] == 1, 'exited'].count()/len(churn_data)*100)

79.60620393796061
20.393796062039378


**Промежуточные выводы**

Наши данные не сбалансированны: на класс "0" приходится 80% данных, а на класс "1" - 20%.

In [9]:
#Столбцы RowNumber — индекс строки в данных, CustomerId — уникальный идентификатор клиента,
#Surname — фамилия в ислледовании не помогут, так что удалим их

clear_churn_data = churn_data.drop(['row_number', 'customer_id', 'surname'], axis=1)

#Обучим модель без учета дисбаланса
data_ohe = pd.get_dummies(clear_churn_data, drop_first=True)
target = data_ohe['exited']
features = data_ohe.drop('exited', axis=1)

features_train, features_ost, target_train, target_ost = train_test_split(features, target, test_size=0.4, \
                                                                          random_state=12345, stratify=target)
features_valid, features_test, target_valid, target_test = train_test_split(features_ost, target_ost, \
                                                                            test_size=0.5, random_state=12345, \
                                                                            stratify=target_ost)
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(5454, 11)
(5454,)
(1818, 11)
(1818,)
(1819, 11)
(1819,)


In [10]:
data_ohe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9091 entries, 0 to 9998
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   credit_score       9091 non-null   int64  
 1   age                9091 non-null   int64  
 2   tenure             9091 non-null   float64
 3   balance            9091 non-null   float64
 4   num_of_products    9091 non-null   int64  
 5   has_cr_card        9091 non-null   int64  
 6   is_active_member   9091 non-null   int64  
 7   estimated_salary   9091 non-null   float64
 8   exited             9091 non-null   int64  
 9   geography_Germany  9091 non-null   uint8  
 10  geography_Spain    9091 non-null   uint8  
 11  gender_Male        9091 non-null   uint8  
dtypes: float64(3), int64(6), uint8(3)
memory usage: 736.9 KB


In [11]:
#Масштабируем данные
numeric = ['credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

features_train.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
9723,-1.292898,-0.660843,0.686341,0.786636,-0.910943,0,0,-0.223548,0,0,1
1224,-1.563714,0.778434,1.034232,0.833795,-0.910943,1,0,1.383021,0,1,0
8377,1.581914,0.106771,-0.357331,-1.222967,0.789359,1,1,-0.308331,0,1,1
8014,0.842379,0.010820,-1.053112,0.807063,-0.910943,1,1,-0.430980,0,1,0
2491,-0.397123,-1.236554,-0.357331,-1.222967,0.789359,1,1,1.392642,0,1,0


In [12]:
#Рассмотрим случайный лес
random_forest_model = None
best_result = 0
best_est = 0
best_depth = 0

for depth in range(1, 100, 5):
    for est in range(1, 100, 5):
        model = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=12345)
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid)
        predicted_valid = model.predict(features_valid)
        if f1_score(target_valid, predicted_valid) > best_result:
            random_forest_model = model
            best_result = f1_score(target_valid, predicted_valid)
            best_depth = depth
            best_est = est
            
print('Глубина:', best_depth)
print('Кол-во деревьев:', best_est)
print("f1:", best_result)


Глубина: 26
Кол-во деревьев: 41
f1: 0.6134185303514377


In [13]:
# Рассмотрим логистическую регрессию
regression_model = None
best_result = 0
best_iter = 0

for i in range(100, 1000, 100):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=i) 
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    predicted_valid = model.predict(features_valid)
    if f1_score(target_valid, predicted_valid) > best_result:
        regression_model = model
        best_result = f1_score(target_valid, predicted_valid)
        best_iter = i
            
print('Кол-во итераций:', best_iter)
print("f1:", best_result)

Кол-во итераций: 100
f1: 0.3306772908366534


**Промежуточный вывод**

Лучший результат показывает модель случайного леса с глубиной 26 и количеством деревьев 41.

In [14]:
#Вычислим значение F1-меры
predicted_valid = random_forest_model.predict(features_valid)
print(f1_score(target_valid, predicted_valid))

#Вычислим значение F1-меры для константной модели
predicted_constant = pd.Series([0]*len(target_valid))
print(f1_score(target_valid, predicted_constant))

0.6134185303514377
0.0


**Промежуточный вывод**

Получили значение F1-меры 0.61.

**Выводы**

Лучший результат показывает модель случайного леса с глубиной 26 и количеством деревьев 41.
Значение F1-меры - 0.61.

## Борьба с дисбалансом

In [15]:
#Сделаем веса классов сбалансированными
model = RandomForestClassifier(random_state=12345, n_estimators=84, max_depth=19, class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5704584040747029


Значение F1-меры составило 0,57.

In [16]:
#Увеличим выборку
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

model = RandomForestClassifier(random_state=12345, n_estimators=84, max_depth=19)

model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.6226138032305433


Значение F1-меры - 0,62.

In [17]:
#Уменьшим выборку
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

model = RandomForestClassifier(random_state=12345, n_estimators=84, max_depth=19)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.6112266112266113


Значение F1-меры - 0,61.

In [18]:
#Уменьшим класс "0" и увеличим класс "1"
def downsample(features, target, fraction, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones] * repeat)
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones] * repeat)
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.5, 2)

model = RandomForestClassifier(random_state=12345, n_estimators=84, max_depth=19)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.6254458977407847


Значение F1-меры - 0,63.

In [19]:
#попробуем изменить порог классификации
model = RandomForestClassifier(random_state=12345, n_estimators=84, max_depth=19)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0.2, 0.4, 0.01):
    predicted_valid = probabilities_one_valid > threshold
    f1 = f1_score(target_valid, predicted_valid)

    print(threshold, f1)

0.2 0.5821782178217823
0.21000000000000002 0.5865580448065173
0.22000000000000003 0.5910509885535901
0.23000000000000004 0.5963791267305645
0.24000000000000005 0.6054644808743169
0.25000000000000006 0.6136618141097424
0.26000000000000006 0.6234357224118315
0.2700000000000001 0.6242774566473988
0.2800000000000001 0.6304604486422668
0.2900000000000001 0.6352657004830917
0.3000000000000001 0.6419753086419753
0.3100000000000001 0.6465408805031446
0.3200000000000001 0.647208121827411
0.3300000000000001 0.65044814340589
0.34000000000000014 0.6475195822454309
0.35000000000000014 0.6453333333333334
0.36000000000000015 0.6531165311653118
0.37000000000000016 0.6602209944751382
0.38000000000000017 0.6574202496532594
0.3900000000000002 0.6448863636363636


При пороге 0.37 значение F1-меры достигло 0.66.

**Промежуточные выводы**

Лучшего результата удалось достичь с помощью изменения порога классификации. F1-мера достигла значения 0.66.

In [20]:
#Вычислим значение AUC-ROC
predicted_valid = probabilities_one_valid > 0.37
auc_roc = roc_auc_score(target_valid, predicted_valid)
auc_roc

0.7827105806790516

**Промежуточные выводы**

Лучшего результата удалось достич с помощью изменения порога классификации. F1-мера достигла значения 0.66.

Значение AUC-ROC составило 0.78.

## Тестирование модели

In [21]:
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
predicted_test = probabilities_one_test > 0.37
print(f1_score(target_test, predicted_test))
roc_auc_score(target_test, model.predict(features_test))

0.6099290780141844


0.7046609506932138

**Промежуточные выводы**

На тестовой выборке значение F1-меры составило 0.61, а значение AUC-ROC - 0.71.

## Выводы

В ходе построения модели предсказания ухода клиентов из банка, была проделана следующая работа:
1. Проведена предобработка данных:
    - Мы устранили пропуски в столбце Tenure - на них приходилось всего 9% данных, поправили названия столбцов. Дубликаты в данных не обнаружены.
2. Провели исследование задачи:
    - Выяснили, что данные не сбалансированны: на класс "0" приходится 80% данных, а на класс "1" - 20%.
    - Исключили столбцы RowNumber, CustomerId, Surname т.к. для обучения модели они не помогут.
    - Провели масштабирование данных.
    - Обучили модель без учета дисбаланса, с использование дамми-ловушки.
    - Рассмотрели 2 модели: случайный лес и логическую регрессию, проверили для каждой модели f1-меру. Лучший результат показала модель случайного леса  с глубиной 26 и количеством деревьев 41 - 0.61, ее мы и ииспользовали в дальнейшем.
3. Проверили разные методы для борьбы с дисбалансом:
    - С использованием метода балансировки весов классов мы получили значение f1-меры равное 0.57.
    - Увеличив выборку с классом "1" в 4 раза мы получили значение f1-меры равное 0.62.
    - Уменьшив выборку с классом "0" в 0.25 раз мы получили значение f1-меры равное 0,61.
    - Увеличив выборку с классом "1" в 2 раза и уменьшив выборку с классом "0" в 2 раза мы получили значение f1-меры равное 0,63.
    - Изменив порог классификации на 0.37 мы получили значение f1-меры равное 0.66. Значение AUC-ROC составило 0.78.
4. Протестировали модель:
    - Результаты предсказаний на тестовой выборке показали значение f1-меры равное 0.61. Также мы получили значение AUC-ROC - 0.71.
В итоге нам удалось достичь значения f1-меры большее 0.59.